In [1]:
# Importar pandas e numpy para trabalhar com Dataframes
import pandas as pd
import numpy as np

# Importar pyarrow para ler arquivo parquet
import pyarrow as pa
import  pyarrow.parquet as pq

In [2]:
# Definir o caminho em que se encontra o dataset em csv com a ordem das questoes da prova
dfItens = pd.read_csv("../../Microdados_enem_2019/DADOS/ITENS_PROVA_2019.csv", sep=';',encoding='iso-8859-1')

In [56]:
# Criar o dicionário modelo contendo os dataframes vazios para cada area
dictSubjects = {"CN":[], "MT":[], "CH":[], "LC":[]}

In [57]:
# Criar o dicionario que será preenchido com a ordem das questoes para cada area
dictGabOrder = dictSubjects.copy()

# Executar as instruções abaixo para cada uma das areas
for subject in dictGabOrder.keys():

    # Iniciar o dataframe vazio auxiliar que será populado com colunas contendo 
    # a ordem das questoes 
    dfGabOrder = pd.DataFrame()
    
    # Filtrar o dataset das questoes para a area e 
    # obter o codigo da prova Azul  
    dfSubject = dfItens[dfItens["SG_AREA"]==subject]
    blueCode = dfSubject[dfSubject["TX_COR"]=="Azul"]["CO_PROVA"].unique()[0]

    # Executar as instruções abaixo caso a area seja 'LC'
    if subject == "LC":

        # Obter os codigos das questoes para a prova azul da area 
        # dividir em duas colunas: codigos considerando 'TP_LINGUA' = 0  e  'TP_LINGUA' = 1
        dfGabOrder["CO_ITEM_0"] = dfSubject[(dfSubject["CO_PROVA"]==blueCode) & (dfSubject["TP_LINGUA"]!=1)][["CO_ITEM"]].reset_index(drop=True)
        dfGabOrder["CO_ITEM_1"] = dfSubject[(dfSubject["CO_PROVA"]==blueCode) & (dfSubject["TP_LINGUA"]!=0)][["CO_ITEM"]].reset_index(drop=True)

        # Obter os valores unicos para a combinação 'CO_PROVA' e 'TP_LINGUA'
        dfUniqueProvaLingua = dfSubject[["CO_PROVA", "TP_LINGUA"]].drop_duplicates().dropna()

        # Executar as instrucoes para cada combinacao unica
        for code, lingua in dfUniqueProvaLingua.itertuples(index=False):
            
            # Converter o codigo da lingua 'TP_LINGUA' em int
            lingua = int(lingua)
            
            # Filtrar o dataset da area pelo codigo da prova e lingua + demais questoes
            # para obter os codigos das questoes e suas posicoes para aquela prova
            dfSubjectFiltered = dfSubject[(dfSubject["CO_PROVA"]==code) & ((dfSubject["TP_LINGUA"]==lingua) | (dfSubject["TP_LINGUA"].isna()))][["CO_ITEM", "CO_POSICAO"]]

            # Converter as posiçoes para o intervalo 0 a 44 e
            # renomear as colunas para que seja identificada o cidogo da lingua
            dfSubjectFiltered["CO_POSICAO"] = dfSubjectFiltered["CO_POSICAO"] - np.min(dfSubjectFiltered["CO_POSICAO"])
            dfSubjectFiltered.rename(columns= {"CO_ITEM":f'CO_ITEM_{lingua}'}, inplace=True) 
            
            # Fazer um merge do dataset fitlrado com o dataset acumulado, 
            # utilizando como chave o codigo da questao
            dfGabOrder = dfGabOrder.merge(dfSubjectFiltered, on=f'CO_ITEM_{lingua}', how="left", suffixes=(None, f'_{code}_{lingua}'))

        # Renomear a primeira coluna para adicionar o codigo da prova e da lingua            
        dfGabOrder.rename(columns= {"CO_POSICAO":f'CO_POSICAO_{blueCode}_{0}'}, inplace=True)

        # Salvar o Dataframe no dicionario 
        dictGabOrder[subject] = dfGabOrder
        

     # Executar as instruções abaixo caso a area não seja 'LC'
    else:
        # Obter os codigos das questoes para a prova azul da area 
        dfGabOrder["CO_ITEM"] = dfSubject[dfSubject["CO_PROVA"]==blueCode][["CO_ITEM"]]

        # Executar as instrucoes para cada coodigo de prova
        for code in dfSubject['CO_PROVA'].unique():

            # Filtrar o dataset da area pelo codigo da prova, para obter
            # os codigos das questoes e suas posicoes para aquela prova
            dfSubjectFiltered = dfSubject[dfSubject["CO_PROVA"]==code][["CO_ITEM", "CO_POSICAO"]]

            # Converter as posiçoes para o intervalo 0 a 44
            dfSubjectFiltered["CO_POSICAO"] = dfSubjectFiltered["CO_POSICAO"] - np.min(dfSubjectFiltered["CO_POSICAO"])
            
            # Fazer um merge do dataset fitlrado com o dataset acumulado, 
            # utilizando como chave o codigo da questao
            dfGabOrder = dfGabOrder.merge(dfSubjectFiltered, on="CO_ITEM", how="left", suffixes=(None, f'_{code}'))
        
         
        # Renomear a primeira coluna para adicionar o codigo da prova e da lingua            
        dfGabOrder.rename(columns= {"CO_POSICAO":f'CO_POSICAO_{blueCode}'}, inplace=True) 

        # Salvar o Dataframe no dicionario 
        dictGabOrder[subject] = dfGabOrder


In [58]:
# Ler os arquivo dos datasets contendo as respostas dos candidatos por area
dictResp = dictSubjects.copy() 

for subject in dictResp.keys():
    dictResp[subject] = pd.read_parquet(f'../../Datasets/MICRODADOS_ENEM_2019_SAOPAULO_BySubject/{subject}.parquet')

In [59]:
# Preencher a primeira posicao da coluna 'CO_POSICAO_522', pois existe um erro no dataset original 
# que nao permitiu a correta identificacao da posicao da questao nessa prova
dictGabOrder["MT"].loc[0,"CO_POSICAO_522"] = 15

# Converter o tipo da coluna para 'int'
dictGabOrder["MT"]["CO_POSICAO_522"] = dictGabOrder["MT"]["CO_POSICAO_522"].astype('int32')

# Remover as linhas dos candidatos que fizeram a prova 519 da area CN
dictResp['CN'] = dictResp['CN'][dictResp['CN']['CO_PROVA_CN'] != 519].reset_index(drop=True)

In [60]:
# Criar o dicionario que será preenchido com os arrays contendo a ordem das 
# questoes para cada linha dos datasets das respostas dos candidatos
dictOrderArrays = dictSubjects.copy()

# Executar as instruções abaixo para cada uma das areas
for subject in dictOrderArrays.keys():
    
    # Iniciar o array vazio 
    arrPositions = []

    # Iniciar o array com o nome das colunas que serão iteradas 
    arrColumnsToIterate = ['NU_INSCRICAO', f'CO_PROVA_{subject}']
   
    # Adicionar a coluna com o codigo da lingua caso a area seja 'LC'
    if subject == 'LC':
        arrColumnsToIterate.append('TP_LINGUA')
 
    # Executar as instrucoes abaixo para cada linha do dataset de respostas
    for index, row in dictResp[subject][arrColumnsToIterate].iterrows():

        # Obter o codigo da prova da linha atual
        codigo = int(row[f'CO_PROVA_{subject}'])

        # Definir o nome da coluna com as posicoes para a prova
        co_posicao = f'CO_POSICAO_{codigo}_{int(row["TP_LINGUA"])}' if subject == 'LC' else f'CO_POSICAO_{codigo}'

        # Converter a coluna com as posicoes da prova em um array
        # e adicionar os demais 'index' que não serão sorteados
        arrPos = np.array(dictGabOrder[subject][co_posicao]) + 1 
        arrPos = np.insert(arrPos, 0 , 0)
        arrPos = np.insert(arrPos, 46 , 46)
        arrPos = np.insert(arrPos, 47 , 47)
        arrPos = np.insert(arrPos, 48 , 48) if subject == 'LC' else arrPos

        # Adicionar o array da linha ao consolidado contendo as demais linhas
        arrPositions.append(arrPos)

    # Salvar o array consolidado no dicionario
    dictOrderArrays[subject] = arrPositions

In [61]:
# Criar o dicionario que será preenchido com as linhas organizadas do
# dataset de respostas
dictRespOrdered = dictSubjects.copy()

# Executar as instruções abaixo para cada uma das areas
for subject in dictRespOrdered.keys():
    
    # Obter o nome das colunas do dataset de respostas
    columns = dictResp[subject].columns

    # Criar um dataframe para cada linha do dataframe de respostas e concatenar
    # em um dataframe unico no final 
    dfSubject = pd.concat([ pd.DataFrame(  [row[dictOrderArrays[subject][index]].values], columns=columns )  for index, row in dictResp[subject].iterrows()], ignore_index=True)
    
    # Salvar o dataframe no dicionario
    dictRespOrdered[subject] = dfSubject

In [ ]:
# Salvar os datasets com as linhas organizadas divididos por area
for subject, dfSubject in dictRespOrdered.items():
    
    table = pa.Table.from_pandas(dfSubject.reset_index(drop=True))

    # Definir onde será salvo o novo dataset filtrado por área
    path = f'../../Datasets/MICRODADOS_ENEM_2019_SAOPAULO_BySubjectOrdered/{subject}.parquet'

    # Gerar um parquet da tabela
    pq.write_to_dataset(
        table,
        root_path=path,
    )